In [1]:
import pandas as pd
import numpy as np
import time
from datetime import timedelta 
from datetime import datetime

In [2]:
# read csv
eb = pd.read_csv('78 EB.csv')
wb = pd.read_csv('78 WB .csv')

# select ramps only
eb = eb[eb.Name.str.contains('to')]
wb = wb[wb.Name.str.contains('to')]

In [3]:
def convtime(s):
    fmt = "%H:%M:%S"
    return datetime.strptime(s, fmt)

# convert datatype of 'time' to datetime 
wb['wb_convtime'] = wb['Time'].apply(convtime)
eb["eb_convtime"] = eb['Time'].apply(convtime)

# take into account the sensor delay time
wb['delay_time'] = wb['wb_convtime'] + timedelta(minutes=1) 

In [4]:
def name_transform(x):
    if 'WB' in x:
        x = x.replace('WB','EB')
    return x

# add a new column to wb to merge wb and eb
wb['transform_name'] = wb['Name'].apply(name_transform)

# merge
new_df = pd.merge(eb, wb, how='inner', left_on=['Name','eb_convtime'], right_on = ['transform_name','delay_time'])

In [5]:
# drop unnecessary columns
new_df = new_df[['Name_x','pcw1occ_x', 'pcw1speed_x', 'pcw1vol_x', 'r_1_offcnt_x', 'r_2_offcnt_x',
                'pcw1occ_y', 'pcw1speed_y', 'pcw1vol_y', 'Psg 1 Cnt_x', 
                 'Psg 2 Cnt_x', 'Psg 3 Cnt_x','r_1_offcnt_y', 'r_2_offcnt_y']]

In [6]:
# convert Nan values to 0
new_df['Psg 1 Cnt_x'] = new_df['Psg 1 Cnt_x'].fillna(0)
new_df['Psg 2 Cnt_x'] = new_df['Psg 2 Cnt_x'].fillna(0)
new_df['Psg 3 Cnt_x'] = new_df['Psg 3 Cnt_x'].fillna(0)
new_df['r_1_offcnt_x'] = new_df['r_1_offcnt_x'].fillna(0)
new_df['r_2_offcnt_x'] = new_df['r_2_offcnt_x'].fillna(0)

In [7]:
# # change column names
# # E: East Bound
# # W: West Bound
# new_df.rename(columns={"Name_x": "name",
#                        "pcw1occ_x": "pcw1occ_E",
#                        "pcw1speed_x": "pcw1speed_E",
#                        "pcw1vol_x": "pcw1vol_x_E",
#                        "r_1_offcnt_x": "r_1_offcnt_E",
#                        "r_2_offcnt_x": "r_2_offcnt_E",
#                        "pcw1occ_y": "pcw1occ_W",
#                        "pcw1speed_y": "pcw1speed_W",
#                        "pcw1vol_y": "pcw1vol_W",
#                        "Psg 1 Cnt_x": "Psg 1 Cnt_E",
#                        "Psg 2 Cnt_x": "Psg 2 Cnt_E",
#                        "Psg 3 Cnt_x": "Psg 2 Cnt_E",
#                        "r_1_offcnt_y": "r_1_offcnt_W",
#                        "r_2_offcnt_y": "r_2_offcnt_W"}, inplace = True)

In [8]:
new_df.head(1)

,Name_x,pcw1occ_x,pcw1speed_x,pcw1vol_x,r_1_offcnt_x,r_2_offcnt_x,pcw1occ_y,pcw1speed_y,pcw1vol_y,Psg 1 Cnt_x,Psg 2 Cnt_x,Psg 3 Cnt_x,r_1_offcnt_y,r_2_offcnt_y
0,Jefferson St to 78 EB,1.1,75,182,0.0,1.0,3.6,64,487,0.0,0.0,0.0,0.0,1.0


# Calculations

In [9]:
N = 2   # number of lanes
TAO = 1.75
FT_MI = 5280 # mile to feet conversion
CONVERSION = 5280/3600

In [48]:
# pass parameter x to probe the x-th row of df
def get_best_rates(x, rates):
    row = new_df.iloc[[x]]  # row we are working with

    # calculate values
    density_up = row['pcw1speed_x']/row['pcw1vol_x']
    q_exit = (row['r_1_offcnt_x']+row['r_2_offcnt_x'])*120

    # density down stream for each rate
    vmerge_vals = dict()   # merge speed
    nds_vals = dict()   # down stream density
    diff = dict()
    for i in rates:
        vols = (((FT_MI*N)*row['pcw1vol_x'])/((TAO*CONVERSION*N*row['pcw1speed_x'])+i))-(14.75/(TAO*CONVERSION))
        vmerge_vals[i] = min(float(vols), float(row['pcw1vol_x'][x]))

        nds_vals[i] = (1/row['pcw1vol_x']) * (row['pcw1speed_x'] + (i/N))-((1/N* q_exit)/vmerge_vals[i])
        
        # take the difference to see which one is closest to nds = 1.15(nups) 
        diff[i] = float(abs(nds_vals[i]-(density_up*1.15)))        

    # get the best rate
    best_rate = min(diff, key=diff.get)
    
    return float(best_rate)

In [49]:
# mu: mean; best_rate
# sigma: standard deviation
def generate_range(mu, sigma = 2):
    return np.random.normal(mu, sigma, 15)

In [50]:
rates = [1008,971,934,896,859,822,785, 748,710,673,636,599,562,524,484]   # initial rates
rates = [i*N for i in rates]   # multiply rates by the number of lanes

In [51]:
for row in range(new_df.size[0]):
    best_rate = get_best_rates(row, rates)
    rates = generate_range(best_rate)

In [ ]:
# what is this for?
# calculation may be wrong. Missing a few components from the formula

# vol_x: upstream
# vol_y: downstream

# For East Bound (variables with x):

density_downstream = []
for i in range(len(new_df['pcw1vol_x'])):
    if new_df['pcw1vol_x'][i] != 0:
        nds = (1/new_df['pcw1vol_x'][i]) * (new_df['pcw1speed_x'][i] + new_df['r_1_offcnt_x'][i])
        # assuming r_1_offcnt_x is ramp, what is ramp speed?
    density_downstream.append(nds)
    
#density_downstream

In [ ]:
# # original code from vince
# diff_df=pd.DataFrame()
# row1 = new_df.iloc[[0]]

# row1['dens_ups']= row1['pcw1speed_x']/row1['pcw1vol_x']
# row1['q_exit'] = (row1['r_1_offcnt_x']+row1['r_2_offcnt_x'])*120
# rates = [1008,971,934,896,859,822,785, 748,710,673,636,599,562,524,484]
# rates2 = []
# # double the rate for two ramps
# for i in rates:
#     rates2.append(i*2)
# # density down stream for each rate
# for i in rates2:
#     vols = (((5280*3)*row1['pcw1vol_x'])/((1.75*1.47*3*row1['pcw1speed_x'])+i))-(14.75-(1.75*1.47))
#     row1['vmerge_'+str(i)] = vols
#     row1['nds_'+str(i)] = (1/row1['pcw1vol_x']) * (row1['pcw1speed_x'] + (i/3))-((.33* row1['q_exit'])/row1['vmerge_'+str(i)])
    
# # take the minimum of upstream volume and eqn
# for i in vmergelst:
#     row1[i] = row1[['pcw1vol_x',i]].min(axis=1)
# # take the difference to see which one is closest to nds = 1.15(nups) 
# for i in rates2:
#     diff_df['diff_'+str(i)]=row1['nds_'+str(i)]-(row1['dens_ups']*1.15)
    
# # get the best rate
# meanloss = []
# for i in diff_df.columns:
#     meanloss.append(np.mean(diff_df[i]))
# best_rate = diff_df.min(axis=0).idxmin()
# best_rate = int(best_rate[5:])
# best_rate

In [ ]:
# # original code from Vince

# row1 = new_df.iloc[[1]]
# row1['dens_ups']= row1['pcw1speed_x']/row1['pcw1vol_x']
# row1['q_ramp'] = (row1['Psg 1 Cnt_x']+row1['Psg 2 Cnt_x']+row1['Psg 3 Cnt_x'])*120
# row1['q_exit'] = (row1['r_1_offcnt_x']+row1['r_2_offcnt_x'])*120
# vmergelst=[]
# # density down stream for each rate
# for i in new_rates:
#     vols = (((5280*3)*row1['pcw1vol_x'])/((1.75*1.47*3*row1['pcw1speed_x'])+i))-(14.75-(1.75*1.47))
#     row1['vmerge_'+str(i)] = vols
#     row1['nds_'+str(i)] = (1/row1['pcw1vol_x']) * (row1['pcw1speed_x'] + (i/3))-((.33* row1['q_exit'])/row1['vmerge_'+str(i)])
#     vmergelst.append('vmerge_'+str(i))
# # take the minimum of upstream volume and eqn
# for i in vmergelst:
#     row1[i] = row1[['pcw1vol_x',i]].min(axis=1)
# # take the difference to see which one is closest to nds = 1.15(nups)
# for i in new_rates:
#     diff2['diff_'+str(i)]=row1['nds_'+str(i)]-(row1['dens_ups']*1.15)
# # get the best rate
# meanloss = []
# for i in diff2.columns:
#     meanloss.append(np.mean(diff2[i]))
# best_rate = diff2.min(axis=0).idxmin()
# best_rate = float(best_rate[5:])
# diff2

In [ ]:
# new_df['dens_ups']= new_df['pcw1speed_x']/new_df['pcw1vol_x']

In [ ]:
# new_df.head()

In [ ]:
# new_df['q_ramp'] = (new_df['Psg 1 Cnt_x']+new_df['Psg 2 Cnt_x']+new_df['Psg 3 Cnt_x'])*120
# new_df['q_exit'] = (new_df['r_1_offcnt_x']+new_df['r_2_offcnt_x'])*120

In [ ]:
#new_df['q_ramp']<new_df['q_exit']

In [ ]:
# rates = [1008,971,934,896,859,822,785, 748,710,673,636,599,562,524,484]
# rates2 = []
# for i in rates:
#     rates2.append(i*2)

In [ ]:
# for i in rates2:
#     vols = (((5280*3)*new_df['pcw1vol_x'])/((1.75*1.47*3*new_df['pcw1speed_x'])+i))-(14.75-(1.75*1.47))
#     new_df['vmerge_'+str(i)] = vols
   
#     #new_df['nds_'+str(i)] = (1/new_df['pcw1vol_x']) * (new_df['pcw1speed_x'] + (i/3))-(.33* new_df['q_exit'])

In [ ]:
# for i in vmergelst:
#     new_df[i] = new_df[['pcw1vol_x',i]].min(axis=1)

In [ ]:
# for i in rates2:
#     new_df['nds_'+str(i)] = (1/new_df['pcw1vol_x']) * (new_df['pcw1speed_x'] + (i/3))-((.33* new_df['q_exit'])/new_df['vmerge_'+str(i)])

In [ ]:
# new_df.head()

In [ ]:
# for i in rates2:
#     diff_df['diff_'+str(i)]=new_df['nds_'+str(i)]-(new_df['dens_ups']*1.15)

In [ ]:
# meanloss = []
# for i in diff_df.columns:
#     meanloss.append(np.mean(diff_df[i]))

In [ ]:
diff_df.head()

In [ ]:
diffs = pd.DataFrame()
diffs['rates'] = rates2
diffs['loss'] = meanloss
diffs

## Normal Distribution Generated Rates

In [ ]:
# mu, sigma = 968, 1 # mean and standard deviation
# new_rates = np.random.normal(mu, sigma, 100)
# new_rates

In [ ]:
# new_diff=pd.DataFrame()
# for i in new_rates:
#     new_diff['diff_'+str(i)]=new_df['nds_'+str(i)]-(new_df['dens_ups']*1.15)

In [ ]:
# for i in new_rates:
#     new_vols = (((5280*3)*new_df['pcw1vol_x'])/((1.75*1.47*3*new_df['pcw1speed_x'])+i))-(14.75-(1.75*1.47))
#     new_df['vmerge_'+str(i)] = new_vols

In [ ]:
# gen = new_df.iloc[:,-100:]

In [ ]:
# gencols = gen.columns

In [ ]:
# gen['pcw1vol_x'] = new_df['pcw1vol_x']
# gen['pcw1speed_x'] = new_df['pcw1speed_x']
# gen['q_exit'] = new_df['q_exit']
# gen['q_exit'] = new_df['dens_ups']

In [ ]:
# gen.describe()

In [ ]:
# for i in gencols:
#     gen[i] = gen[['pcw1vol_x',i]].min(axis=1)

In [ ]:
# for i in new_rates:
#     gen['nds_'+str(i)] = (1/new_df['pcw1vol_x']) * (new_df['pcw1speed_x'] + (i/3))-((.33* new_df['q_exit'])/new_df['vmerge_'+str(i)])

In [ ]:
# gen.describe()

In [ ]:
# import statistics 

In [ ]:
# gen_diff = pd.DataFrame()

In [ ]:
# for i in new_rates:
#     gen_diff['diff_'+str(i)]=gen['nds_'+str(i)]-(new_df['dens_ups']*1.15)

In [ ]:
# gen_diff

In [ ]:
# gen_diff.describe()

In [ ]:
# gen_meanloss = []
# for i in gen_diff.columns:
#     gen_meanloss.append(np.mean(gen_diff[i]))

In [ ]:
# gendiffs = pd.DataFrame()
# gendiffs['rates'] = new_rates
# gendiffs['loss'] = gen_meanloss
# gendiffs

In [ ]:
# gendiffs['loss'].min()

In [ ]:
# filtered_df = new_df[new_df[['Name_x', 'transform_name']].notnull()] #this makes all values NaN?

In [ ]:
# filtered_df

In [ ]:
# q_exit = (filtered_df.r_1_offcnt_x+filtered_df.r_1_offcnt_y)*120
# q_exit.unique()